In [1]:
!git clone https://github.com/TienDat57/Thesis.git

Cloning into 'Thesis'...
remote: Enumerating objects: 757, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 757 (delta 50), reused 47 (delta 27), pack-reused 679
Receiving objects: 100% (757/757), 195.70 MiB | 10.59 MiB/s, done.
Resolving deltas: 100% (516/516), done.


In [2]:
!pip install transformers
!pip install seqeval
!pip install Keras-Preprocessing
!pip install tensorflow<2.16,>=2.15
!pip install --upgrade jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=9e50b3c85b455c88c4a1c238f8f684ad3110536be5a56405be16d372e348a1f0
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00
/bin/bash: line 1: 2.16,: No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.26+cuda12.cudnn89
    Uninstalling jaxlib-0.4.26+cuda12.cudnn89:
      Successfully uninstalled jaxlib-0.4.26+cuda12.cudnn89
  Attempting uninstall: jax
    Found existing installation: jax 0.4.26
    Uninstalling jax-0.4.26:
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd /content/Thesis/src/model/SRL_Single_Task

# Masked Data Creator

In [ ]:
from generate_data_masked import MaskedDataCreator
import torch

In [ ]:
# paste saved model path to here with 4 model task: SRL, SRL+depency, SRL+constituency, SRL+depency+constituency
saved_model_path = '/content/drive/MyDrive/Thesis/result/srl/multi_task_model_19_15318.pt'

In [ ]:
# paste saved tokenizer path to here
path_origin = '/content/Thesis/src/model/SRL_Single_Task/ner_make_masked/ner_coNLL_srl_dev_abolish.tsv'
path_cons = '/content/drive/MyDrive/Thesis/masked/cons/ner_coNLL_srl_dev_abolish_cons.tsv'
path_depen = '/content/Thesis/src/model/SRL_Single_Task/ner_make_masked/ner_coNLL_srl_dev_abolish_depen.tsv'
path_predicate = '/content/Thesis/src/model/SRL_Single_Task/ner_make_masked/ner_coNLL_srl_dev_abolish_predicate.tsv'

In [ ]:
masked_data_creator = MaskedDataCreator(origin_path_tsv=path_origin,
                                        saved_model_path=saved_model_path)

In [ ]:
origin_data = masked_data_creator.infer_data(path_data=path_origin)
predicate_data = masked_data_creator.infer_data(path_data=path_predicate)
depen_data = masked_data_creator.infer_data(path_data=path_depen)
cons_data = masked_data_creator.infer_data(path_data=path_cons)

In [ ]:
from ast import literal_eval
import pandas as pd

def group_elements_with_label(labels, indexChange, is_depen=True):
    groups = {}
    current_group_label = None
    current_group = []
    for i, label in enumerate(labels):
        if label.startswith('B-A') or label.startswith('I-A') or label == 'B-V':
            if current_group:
                groups.setdefault(current_group_label, []).extend(current_group)
            if label == 'B-V':
                current_group_label = label
            else:
                current_group_label = label[2:]
            current_group = [indexChange[i]] if is_depen else [i]
        else:
            if current_group_label and current_group:
                groups.setdefault(current_group_label, []).extend(current_group)
            current_group_label = None
            current_group = []
    if current_group_label and current_group:
        groups.setdefault(current_group_label, []).extend(current_group)
    return groups

def find_differences(origin_dict, compared_dict):
    different_values = {}
    for key in set(origin_dict.keys()) & set(compared_dict.keys()):
        if origin_dict[key] != compared_dict[key]:
            if key != 'B-V' and len(origin_dict[key]) != len(compared_dict[key]):
                different_values[key] = list(set(origin_dict[key]) ^ set(compared_dict[key]))
    return list(different_values.keys()) if different_values else None

def load_data_tsv(path_tsv):
    allData = []
    for line in open(path_tsv):
        cols = line.strip("\n").split("\t")
        assert len(cols) == 3, "Data not in NER format"
        row = {"uid":cols[0], "label":literal_eval(cols[1]), "sentence":literal_eval(cols[2])}
        assert type(row['label'])==list, "Label should be in list of token labels format in data"
        allData.append(row)
    return allData

def get_list_data_masked_cons(data_tsv_depen, list_data_tsv_cons):
    result = []
    for idx, cons_sentence in enumerate(list_data_tsv_cons):
        if data_tsv_depen['sentence'] != cons_sentence['sentence'] and data_tsv_depen['uid'] == cons_sentence['uid']:
            result.append(idx)
    return result

def get_idx_head(data_tsv_origin, data_tsv_to_compare):
    min_len = min(len(data_tsv_origin['sentence']), len(data_tsv_to_compare['sentence']))
    for i in range(min_len):
        if data_tsv_origin['sentence'][i] != data_tsv_to_compare['sentence'][i]:
            return i
    return None

In [ ]:
def create_masked_data_cons(origin_data, depen_data, cons_data):
    allIdsOrigin, allLabelsOrigin, _, allIndexChangeOrigin, allSequenceOutputOrigin = origin_data
    allIdsCons, allLabelsCons, _, allIndexChangeCons, allSequenceOutputCons = cons_data
    allIdsDepen, allLabelsDepen, _, allIndexChangeDepen, allSequenceOutputDepen = depen_data

    data_tsv_origin = load_data_tsv(path_origin)
    data_tsv_depen = load_data_tsv(path_depen)
    data_tsv_cons = load_data_tsv(path_cons)

    data_tsv_masked = []

    masked_data = []
    jump_count = 0

    for i in range(len(allIdsOrigin)):
        elements_origin = group_elements_with_label(allLabelsOrigin[i], allIndexChangeOrigin[i])
        elements_origin_ = group_elements_with_label(allLabelsOrigin[i], allIndexChangeOrigin[i], is_depen=False)

        for j in range(jump_count, len(allIdsDepen)):
            words_embedding_data_masked = torch.tensor(allSequenceOutputOrigin[i])
            if allIdsDepen[j] == allIdsOrigin[i]:
                idx_masked_cons = get_list_data_masked_cons(data_tsv_depen[j], data_tsv_cons)
                elements_depen = group_elements_with_label(allLabelsDepen[j], allIndexChangeDepen[j])
                is_different = find_differences(elements_origin, elements_depen)
                if is_different and len(is_different) == 1:
                    assert len(elements_origin['B-V']) == 1, 'Predicate must be unique'
                    idx_head_in_org = get_idx_head(data_tsv_origin[i], data_tsv_depen[j])
                    all_idx_change_depen_copy = allIndexChangeDepen[j][:]

                    all_idx_change_depen_copy.insert(idx_head_in_org, -1)

                    elements_cons = group_elements_with_label(allLabelsCons[j], allIndexChangeCons[j])

                    arg_change_in_cons = find_differences(elements_origin, elements_cons)

                    words_embedding_data_masked[allIndexChangeOrigin[i][idx_head_in_org]] = torch.tensor(allSequenceOutputCons[j][elements_cons[arg_change_in_cons[0]][0]])

                    # assign cons masked
                    lst_idx_change_cons = []
                    for idx_change in elements_origin[arg_change_in_cons[0]]:
                        if idx_change != allIndexChangeOrigin[i][idx_head_in_org]:
                            lst_idx_change_cons.append(idx_change)

                    for idx_ in range(len(lst_idx_change_cons)):
                        words_embedding_data_masked[lst_idx_change_cons[idx_]] = torch.tensor(allSequenceOutputDepen[j][elements_depen[arg_change_in_cons[0]][idx_]])

                    data_tsv_masked.append(data_tsv_origin[i])

                    masked_data.append({
                        'masked_data': words_embedding_data_masked,
                    })
            else:
                jump_count = j
                break
    df = pd.DataFrame(data_tsv_masked)
    df.to_csv(path_origin.replace('.tsv', '_masked_cons.tsv'), sep='\t', index=False, header=False)
    return masked_data

In [ ]:
masked_data_cons = create_masked_data_cons(origin_data, depen_data, cons_data)
masked_data = torch.stack([torch.tensor(item['masked_data']) for item in masked_data_cons], dim=0)
print(masked_data.size())

In [ ]:
masked_data_depen = masked_data_creator.create_masked_data_depen(origin_data, predicate_data, depen_data)
masked_data = torch.stack([torch.tensor(item['masked_data']) for item in masked_data_depen], dim=0)
print(masked_data.size())

torch.Size([207, 128, 768])


<ipython-input-89-b234df1e5102>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masked_data = torch.stack([torch.tensor(item['masked_data']) for item in masked_data_depen], dim=0)


# Assign Word Embedding

In [ ]:
from make_data_masked import WordEmbeddingAssigner

In [ ]:
wordAssigner = WordEmbeddingAssigner(pred_file_path=path_origin,
                                     out_dir='./pathToPredict',
                                     has_labels=True,
                                     task_name='conllsrl',
                                     saved_model_path=saved_model_path,
                                     masked_data=masked_data)

In [ ]:
labMap = wordAssigner.get_labelMap()
labMap = {v: k for k, v in labMap.items()}

In [ ]:
wordAssigner.load_and_create_data(isMasked=True, masked_data_type='depen')
# wordAssigner.load_and_create_data(isMasked=True, masked_data_type='cons') # uncomment this line if you want to create masked data for constituency task

In [ ]:
resultwordMasked = wordAssigner.assign_word_embedding_data()
resultwordMasked = {
    'uids': resultwordMasked[0],
    'labels': resultwordMasked[1],
    'preds': resultwordMasked[2],
    'scores': resultwordMasked[3],
    'logits': resultwordMasked[4],
    'sequence_output': resultwordMasked[5],
    'index_change': resultwordMasked[6]
}

In [ ]:
wordAssigner.load_and_create_data()

In [ ]:
resultwordOrigin = wordAssigner.assign_word_embedding_data()
resultwordOrigin = {
    'uids': resultwordOrigin[0],
    'labels': resultwordOrigin[1],
    'preds': resultwordOrigin[2],
    'scores': resultwordOrigin[3],
    'logits': resultwordOrigin[4],
    'sequence_output': resultwordOrigin[5],
    'index_change': resultwordOrigin[6]
}

In [ ]:
def get_idx_arg_preds(preds_origin, preds_masked, label_origin=None):
    list_idx_arg_change = []
    assert len(preds_origin) == len(preds_masked), 'Length of preds_origin and preds_masked must be the same'
    for i in range(len(preds_origin)):
        test = preds_origin[i].startswith('B-A') or preds_origin[i].startswith('I-A') or preds_masked[i].startswith('B-A') or preds_masked[i].startswith('I-A')
        if label_origin:
            assert len(preds_origin) == len(label_origin), 'Length of preds_origin and label_origin must be the same'
            if test or label_origin[i].startswith('B-A') or label_origin[i].startswith('I-A'):
                list_idx_arg_change.append(i)
        else:
            if test:
                list_idx_arg_change.append(i)
    return list_idx_arg_change

def get_idx_labels_arg(labels):
    list_idx_label_arg = []
    for i in range(len(labels)):
        if labels[i].startswith('B-A') or labels[i].startswith('I-A'):
            list_idx_label_arg.append(i)
    return list_idx_label_arg

def calculateInfluenceScore(outLogitsSigmoid_original, outLogitsSigmoid_meddle, list_arg_change):
    influence_score = []
    weight = []
    assert len(outLogitsSigmoid_original) == len(outLogitsSigmoid_meddle)
    for i in range(len(outLogitsSigmoid_original)):
        if i not in list_arg_change:
            continue
        max_index_original = torch.argmax(outLogitsSigmoid_original[i])
        max_index_meddle = torch.argmax(outLogitsSigmoid_meddle[i])
        if max_index_original == max_index_meddle:
            influence_score.append((outLogitsSigmoid_original[i][max_index_original] - outLogitsSigmoid_meddle[i][max_index_meddle]) / max(outLogitsSigmoid_original[i][max_index_original], outLogitsSigmoid_meddle[i][max_index_meddle]))
            weight.append(1)
        else:
            influ_old_label = (outLogitsSigmoid_original[i][max_index_original] - outLogitsSigmoid_meddle[i][max_index_original]) / max(outLogitsSigmoid_original[i][max_index_original], outLogitsSigmoid_meddle[i][max_index_original])
            influ_new_label = (outLogitsSigmoid_meddle[i][max_index_meddle] - outLogitsSigmoid_original[i][max_index_meddle]) / max(outLogitsSigmoid_original[i][max_index_meddle], outLogitsSigmoid_meddle[i][max_index_meddle])
            influence_score.append(influ_old_label + influ_new_label)
            weight.append(2)
    return influence_score, weight



In [ ]:
import torch
for i in range(len(resultwordOrigin['logits'])):
    resultwordOrigin['logits'][i] = torch.tensor(resultwordOrigin['logits'][i])
for i in range(len(resultwordMasked['logits'])):
    resultwordMasked['logits'][i] = torch.tensor(resultwordMasked['logits'][i])

influenceScore, weight = calculateInfluenceScore(resultwordOrigin['logits'][0], resultwordMasked['logits'][0], get_idx_arg_preds(resultwordOrigin['preds'][0], resultwordMasked['preds'][0]))

In [ ]:
print(sum(influenceScore) / sum(weight))

# Calculate Relevance

In [ ]:
def relevance_score(prob_origin_, prob_masked_, label_gold, label_origin, label_masked):
    relevance = []
    weight = []
    jud_space = get_idx_arg_preds(label_origin, label_masked, label_gold)
    for i in range(len(prob_origin_)):
        if i not in jud_space:
            continue
        max_index_origin = torch.argmax(prob_origin_[i])
        max_index_masked = torch.argmax(prob_masked_[i])
        idx_label_gold = labMap[str(label_gold[i])]
        if label_gold[i] == label_origin[i] and label_gold[i] == label_masked[i]:
            score_increase_gold = (prob_origin_[i][idx_label_gold] - prob_masked_[i][idx_label_gold])/max(prob_origin_[i][idx_label_gold], prob_masked_[i][idx_label_gold])
            relevance.append(score_increase_gold)
            weight.append(1)
        elif label_origin[i] != label_masked[i] and label_origin[i] == label_gold[i]:
            score_increase_gold = (prob_origin_[i][idx_label_gold] - prob_masked_[i][idx_label_gold])/max(prob_origin_[i][idx_label_gold], prob_masked_[i][idx_label_gold])
            score_decrease_mask = (prob_masked_[i][max_index_masked] - prob_origin_[i][max_index_masked])/max(prob_masked_[i][max_index_masked], prob_origin_[i][max_index_masked])
            relevance.append((score_increase_gold + score_decrease_mask)/2)
            weight.append(2)
        elif label_origin[i] != label_masked[i] and label_masked[i] == label_gold[i]:
            score_increase_gold = (prob_origin_[i][idx_label_gold] - prob_masked_[i][idx_label_gold])/max(prob_origin_[i][idx_label_gold], prob_masked_[i][idx_label_gold])
            score_decrease_origin = (prob_masked_[i][max_index_origin] - prob_origin_[i][max_index_origin])/max(prob_masked_[i][max_index_origin], prob_origin_[i][max_index_origin])
            relevance.append((score_increase_gold + score_decrease_origin)/2)
            weight.append(2)
        elif label_gold[i] != label_origin[i] and label_masked[i] != label_gold[i]:
            score_increase_gold = (prob_origin_[i][idx_label_gold] - prob_masked_[i][idx_label_gold])/max(prob_origin_[i][idx_label_gold], prob_masked_[i][idx_label_gold])
            score_decrease_mask = (prob_masked_[i][max_index_masked] - prob_origin_[i][max_index_masked])/max(prob_masked_[i][max_index_masked], prob_origin_[i][max_index_masked])
            score_decrease_origin = (prob_masked_[i][max_index_origin] - prob_origin_[i][max_index_origin])/max(prob_masked_[i][max_index_origin], prob_origin_[i][max_index_origin])
            relevance.append((score_increase_gold + score_decrease_mask + score_decrease_origin)/3)
            weight.append(1)
    return relevance, weight

In [ ]:
relevance_in_all_words, weights = relevance_score(resultwordOrigin['logits'][0], resultwordMasked['logits'][0], resultwordOrigin['labels'][0], resultwordOrigin['preds'][0], resultwordMasked['preds'][0])

In [ ]:
sum(relevance_in_all_words) / sum(weights)

In [ ]:
def competence_score(origin, masked):
    result = []
    jump_count = 0
    for i in range(len(origin['uids'])):
        for j in range(jump_count, len(masked['uids'])):
            if origin['uids'][i] == masked['uids'][j]:
                influenceScore, weight_influ = calculateInfluenceScore(origin['logits'][i], masked['logits'][j], get_idx_arg_preds(origin['preds'][i], masked['preds'][j]))
                relevance_in_all_words, weight_relevance = relevance_score(origin['logits'][i], masked['logits'][j], origin['labels'][i], origin['preds'][i], masked['preds'][j])
                result.append(
                    {
                        'uid': masked['uids'][j],
                        'influence': sum(influenceScore) / sum(weight_influ),
                        'relevance': sum(relevance_in_all_words) / sum(weight_relevance)
                    }
                )
            else :
                jump_count = j
                break
    return result

In [ ]:
competence = competence_score(resultwordOrigin, resultwordMasked)

In [ ]:
from scipy.stats import spearmanr, permutation_test

influence_values = [item['influence'] for item in competence]
relevance_values = [item['relevance'] for item in competence]

# Calculate Spearman correlation coefficient
correlation_coefficient, p_value = spearmanr(influence_values, relevance_values)


print("Spearman correlation coefficient:", correlation_coefficient)
print("p-value:", p_value)
print('saved model path', saved_model_path)

In [ ]:
import matplotlib.pyplot as plt

influence = [item['influence'] for item in competence]
relevance = [item['relevance'] for item in competence]

# line chart
plt.plot(influence, label='influence')
plt.plot(relevance, label='relevance')
plt.legend()
plt.show()
